<h1> 1. Exploring natality dataset </h1>

This notebook illustrates:
<ol>
<li> Exploring a BigQuery dataset using Datalab
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'cs-350-elena-manilich'   # CHANGE this to a globally unique value. Your project name is a good option to try.
PROJECT = 'cs-350-elena-manilich'     # CHANGE this to your project name
REGION = 'us-central1'               # CHANGE this to one of the regions supported by Cloud AI Platform https://cloud.google.com/ml-engine/docs/tensorflow/regions

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [3]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

In [4]:
query = """
SELECT *
FROM
  `data-to-insights.ecommerce.all_sessions_raw`
LIMIT 10
"""

In [5]:
# Call BigQuery and examine in dataframe
from google.cloud import bigquery
df = bigquery.Client().query(query ).to_dataframe()
df.head()

,fullVisitorId,channelGrouping,time,country,city,totalTransactionRevenue,transactions,timeOnSite,pageviews,sessionQualityDim,...,itemQuantity,itemRevenue,transactionRevenue,transactionId,pageTitle,searchKeyword,pagePathLevel1,eCommerceAction_type,eCommerceAction_step,eCommerceAction_option
0,4685359868372583247,Direct,0,Brazil,not available in demo dataset,None,None,None,1,NaN,...,None,None,None,None,Store search results,None,/asearch.html,0,1,None
1,7904482661507817320,Organic Search,0,Philippines,not available in demo dataset,None,None,None,1,NaN,...,None,None,None,None,Bags | Google Merchandise Store,None,/google+redesign/,0,1,None
2,5461242285985669320,Organic Search,0,United States,New York,None,None,None,1,NaN,...,None,None,None,None,Bags | Google Merchandise Store,None,/google+redesign/,0,1,None
3,0616948764493788763,Organic Search,0,Germany,not available in demo dataset,None,None,None,1,NaN,...,None,None,None,None,Bags | Google Merchandise Store,None,/google+redesign/,0,1,None
4,9847948748488330998,Organic Search,0,United Kingdom,London,None,None,None,1,NaN,...,None,None,None,None,Bags,None,/google+redesign/,0,1,None


<h2> Explore data </h2>
Write a query to find out the number of record in the ecommerce dataset.

In [11]:
# Create SQL query
query = """
SELECT Count(*) as NumRecords
FROM
  `data-to-insights.ecommerce.all_sessions_raw`
LIMIT 10
"""

In [12]:
# Call BigQuery and examine in dataframe
from google.cloud import bigquery
df = bigquery.Client().query(query).to_dataframe()
df.head()

,NumRecords
0,21552195


<h2> Explore data </h2>
Write a query to find out the number of unique channelGrouping